In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'distrivite.settings.dev')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [10]:
from apps.users.models import User
from apps.organization.models import Organization
from django.db.models import When, BooleanField, Value, Case

In [11]:
user = User.objects.filter(email="test@test.com").first()

In [12]:
teaching_organizations = Organization.objects.filter(employees__organization_user__user=user).values("id")
studying_organizations = Organization.objects.filter(students__user=user).values("id")
working_organizations = Organization.objects.filter(organization_users__user=user).values("id")

In [13]:
print(studying_organizations)

<QuerySet [{'id': UUID('b9e036b6-d1ea-4f91-8330-2b5acdf291f7')}]>


In [14]:
all_organizations = [*studying_organizations, *working_organizations, *teaching_organizations]

In [16]:
organization_queryset = Organization.objects.filter(pk__in=[organization["id"] for organization in all_organizations]).distinct()

In [17]:

# Annotate the queryset with the additional attributes
organization_queryset = organization_queryset.annotate(
    is_student=Case(
        When(id__in=studying_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_staff=Case(
        When(id__in=working_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_teacher=Case(
        When(id__in=teaching_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    )
)

In [20]:
for organization in organization_queryset:
    print(organization.pk, organization.name, organization.is_staff, organization.is_teacher, organization.is_student)

03ecdccf-72b3-4ec6-b7e3-605ae85f700e Institut la cote True False False
0b1224f0-7e44-4b9e-a4d7-8cdf06c04314 INSTITUT SUPÉRIEUR DOUALA MANGA BELL True True False
0f4a0107-df3a-4a12-86ba-055870cf7d50 Institut de la montagne True False False
3100e3a3-dc55-4b14-b75b-f6187b99c6b6 Institut des leader True False False
533c1638-d7e3-48b0-9011-df1a0ebd3a9e Institut universitaire du golfe de guinéé True False False
a2f600a2-bd07-4716-af2f-581e4a2f1296 Institut superieur Nanfack True False False
a565dd37-7a77-44c0-bd67-b054ba6048ed creditium True False False
b9e036b6-d1ea-4f91-8330-2b5acdf291f7 INSTITUT SUPÉRIEUR DOUALA MANGA-BELL True True True


In [ ]:
teaching_organizations = Organization.objects.filter(employees__organization_user__user=user).values("id")
studying_organizations = Organization.objects.filter(students__user=user).values("id")
working_organizations = Organization.objects.filter(organization_users__user=user).values("id")

all_organizations = [*studying_organizations, *working_organizations, *teaching_organizations]
organization_queryset = Organization.objects.filter(pk__in=[organization["id"] for organization in all_organizations]).distinct()

# Annotate the queryset with the additional attributes
organization_queryset = organization_queryset.annotate(
    is_student=Case(
        When(id__in=studying_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_staff=Case(
        When(id__in=working_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_teacher=Case(
        When(id__in=teaching_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    )
)

In [26]:
from django.db.models import Case, Value, BooleanField

user = User.objects.filter(email="test@test.com").first()

# Use values_list to obtain flat lists of organization IDs
teaching_organizations = Organization.objects.filter(employees__organization_user__user=user).values_list("id", flat=True)
studying_organizations = Organization.objects.filter(students__user=user).values_list("id", flat=True)
working_organizations = Organization.objects.filter(organization_users__user=user).values_list("id", flat=True)

# Use the | operator for queryset union
all_organizations = studying_organizations | working_organizations | teaching_organizations

# Filter organizations by IDs and make it distinct
organization_queryset = Organization.objects.filter(pk__in=all_organizations).distinct()

# Annotate the queryset with the additional attributes
organization_queryset = organization_queryset.annotate(
    is_student=Case(
        When(id__in=studying_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_staff=Case(
        When(id__in=working_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    ),
    is_teacher=Case(
        When(id__in=teaching_organizations, then=Value(True)),
        default=Value(False),
        output_field=BooleanField()
    )
)

print(organization_queryset)

<QuerySet [<Organization: Institut la cote>, <Organization: INSTITUT SUPÉRIEUR DOUALA MANGA BELL>, <Organization: Institut de la montagne>, <Organization: Institut des leader>, <Organization: Institut universitaire du golfe de guinéé>, <Organization: Institut superieur Nanfack>, <Organization: creditium>, <Organization: INSTITUT SUPÉRIEUR DOUALA MANGA-BELL>]>
